In [1]:
import pandas as pd
import numpy as np


In [ ]:
# Path to your large CSV file
# This is the file your created by merging all the yearly data(12 months) using the “MergeCSV.ipynb” notebook.
file_path = 'Your CSV file.csv'

# Load the CSV file in chunks
chunk_size = 10**6  # Adjust chunk size as needed
csv_reader = pd.read_csv(file_path, chunksize=chunk_size)

# Placeholder for monthly data frames
monthly_data = {month: [] for month in range(1, 13)}
total_counts = {month: 0 for month in range(1, 13)}

# Process each chunk
for chunk in csv_reader:
    # Assuming the CSV has a 'started_at' column in datetime format
    chunk['started_at'] = pd.to_datetime(chunk['started_at'])
    for month in range(1, 13):
        monthly_chunk = chunk[chunk['started_at'].dt.month == month]
        if not monthly_chunk.empty:
            monthly_data[month].append(monthly_chunk)
            total_counts[month] += len(monthly_chunk)

# Combine all chunks for each month into one DataFrame per month
for month in range(1, 13):
    if monthly_data[month]:
        monthly_data[month] = pd.concat(monthly_data[month])

# Calculate total size of the combined data
total_size = sum(df.memory_usage(deep=True).sum() for df in monthly_data.values() if isinstance(df, pd.DataFrame))

# Determine the sample size for each month
sample_sizes = {month: int((count / total_size) * (900 * 10**6)) for month, count in total_counts.items()}

# Placeholder for the sampled data
sampled_data = []

# Sample data proportionately
for month in range(1, 13):
    if isinstance(monthly_data[month], pd.DataFrame):
        sample_size = min(len(monthly_data[month]), sample_sizes[month])
        sampled_data.append(monthly_data[month].sample(n=sample_size, random_state=1))

# Combine all sampled data into one DataFrame
final_sample_df = pd.concat(sampled_data)

# Save the sampled data into one CSV file
final_sample_df.to_csv('your sample file here.csv', index=False)